---
# Model T - Transfer Learning, Data Augmentation, Fine Tuning, Adam
TODO:

---
#### Imports and Setup

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Suppressing tensorflow errors
import tensorflow as tf
print(f'TensorFlow version: {tf.__version__}')
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)
import matplotlib.pyplot as plt
import pickle
import numpy as np
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import callbacks, layers, optimizers, models
from keras import regularizers
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay ,accuracy_score, precision_score, recall_score, f1_score
from PIL import Image

---
#### Resizing the images for the VGG16 model

In [ ]:
IMG_SIZE = 128

def resize_image(image_path, new_dimensions=(IMG_SIZE, IMG_SIZE)):
    with Image.open(image_path) as img:
        resized_img = img.resize(new_dimensions)
    return resized_img

def resize_images_in_directory(input_directory, output_directory, new_dimensions=(IMG_SIZE, IMG_SIZE)):
    os.makedirs(output_directory, exist_ok=True)
    for filename in os.listdir(input_directory):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(input_directory, filename)
            output_path = os.path.join(output_directory, filename)
            resized_img = resize_image(image_path, new_dimensions)
            resized_img.save(output_path)

resize_images_in_directory('../data/test/000_airplane', f'../data/test_resized_{IMG_SIZE}/000_airplane')
resize_images_in_directory('../data/test/001_automobile', f'../data/test_resized_{IMG_SIZE}/001_automobile')
resize_images_in_directory('../data/test/002_bird', f'../data/test_resized_{IMG_SIZE}/002_bird')
resize_images_in_directory('../data/test/003_cat', f'../data/test_resized_{IMG_SIZE}/003_cat')
resize_images_in_directory('../data/test/004_deer', f'../data/test_resized_{IMG_SIZE}/004_deer')
resize_images_in_directory('../data/test/005_dog', f'../data/test_resized_{IMG_SIZE}/005_dog')
resize_images_in_directory('../data/test/006_frog', f'../data/test_resized_{IMG_SIZE}/006_frog')
resize_images_in_directory('../data/test/007_horse', f'../data/test_resized_{IMG_SIZE}/007_horse')
resize_images_in_directory('../data/test/008_ship', f'../data/test_resized_{IMG_SIZE}/008_ship')
resize_images_in_directory('../data/test/009_truck', f'../data/test_resized_{IMG_SIZE}/009_truck')
resize_images_in_directory('../data/train1/000_airplane', f'../data/train1_resized_{IMG_SIZE}/000_airplane')
resize_images_in_directory('../data/train1/001_automobile', f'../data/train1_resized_{IMG_SIZE}/001_automobile')
resize_images_in_directory('../data/train1/002_bird', f'../data/train1_resized_{IMG_SIZE}/002_bird')
resize_images_in_directory('../data/train1/003_cat', f'../data/train1_resized_{IMG_SIZE}/003_cat')
resize_images_in_directory('../data/train1/004_deer', f'../data/train1_resized_{IMG_SIZE}/004_deer')
resize_images_in_directory('../data/train1/005_dog', f'../data/train1_resized_{IMG_SIZE}/005_dog')
resize_images_in_directory('../data/train1/006_frog', f'../data/train1_resized_{IMG_SIZE}/006_frog')
resize_images_in_directory('../data/train1/007_horse', f'../data/train1_resized_{IMG_SIZE}/007_horse')
resize_images_in_directory('../data/train1/008_ship', f'../data/train1_resized_{IMG_SIZE}/008_ship')
resize_images_in_directory('../data/train1/009_truck', f'../data/train1_resized_{IMG_SIZE}/009_truck')
resize_images_in_directory('../data/train2/000_airplane', f'../data/train2_resized_{IMG_SIZE}/000_airplane')
resize_images_in_directory('../data/train2/001_automobile', f'../data/train2_resized_{IMG_SIZE}/001_automobile')
resize_images_in_directory('../data/train2/002_bird', f'../data/train2_resized_{IMG_SIZE}/002_bird')
resize_images_in_directory('../data/train2/003_cat', f'../data/train2_resized_{IMG_SIZE}/003_cat')
resize_images_in_directory('../data/train2/004_deer', f'../data/train2_resized_{IMG_SIZE}/004_deer')
resize_images_in_directory('../data/train2/005_dog', f'../data/train2_resized_{IMG_SIZE}/005_dog')
resize_images_in_directory('../data/train2/006_frog', f'../data/train2_resized_{IMG_SIZE}/006_frog')
resize_images_in_directory('../data/train2/007_horse', f'../data/train2_resized_{IMG_SIZE}/007_horse')
resize_images_in_directory('../data/train2/008_ship', f'../data/train2_resized_{IMG_SIZE}/008_ship')
resize_images_in_directory('../data/train2/009_truck', f'../data/train2_resized_{IMG_SIZE}/009_truck')
resize_images_in_directory('../data/train3/000_airplane', f'../data/train3_resized_{IMG_SIZE}/000_airplane')
resize_images_in_directory('../data/train3/001_automobile', f'../data/train3_resized_{IMG_SIZE}/001_automobile')
resize_images_in_directory('../data/train3/002_bird', f'../data/train3_resized_{IMG_SIZE}/002_bird')
resize_images_in_directory('../data/train3/003_cat', f'../data/train3_resized_{IMG_SIZE}/003_cat')
resize_images_in_directory('../data/train3/004_deer', f'../data/train3_resized_{IMG_SIZE}/004_deer')
resize_images_in_directory('../data/train3/005_dog', f'../data/train3_resized_{IMG_SIZE}/005_dog')
resize_images_in_directory('../data/train3/006_frog', f'../data/train3_resized_{IMG_SIZE}/006_frog')
resize_images_in_directory('../data/train3/007_horse', f'../data/train3_resized_{IMG_SIZE}/007_horse')
resize_images_in_directory('../data/train3/008_ship', f'../data/train3_resized_{IMG_SIZE}/008_ship')
resize_images_in_directory('../data/train3/009_truck', f'../data/train3_resized_{IMG_SIZE}/009_truck')
resize_images_in_directory('../data/train4/000_airplane', f'../data/train4_resized_{IMG_SIZE}/000_airplane')
resize_images_in_directory('../data/train4/001_automobile', f'../data/train4_resized_{IMG_SIZE}/001_automobile')
resize_images_in_directory('../data/train4/002_bird', f'../data/train4_resized_{IMG_SIZE}/002_bird')
resize_images_in_directory('../data/train4/003_cat', f'../data/train4_resized_{IMG_SIZE}/003_cat')
resize_images_in_directory('../data/train4/004_deer', f'../data/train4_resized_{IMG_SIZE}/004_deer')
resize_images_in_directory('../data/train4/005_dog', f'../data/train4_resized_{IMG_SIZE}/005_dog')
resize_images_in_directory('../data/train4/006_frog', f'../data/train4_resized_{IMG_SIZE}/006_frog')
resize_images_in_directory('../data/train4/007_horse', f'../data/train4_resized_{IMG_SIZE}/007_horse')
resize_images_in_directory('../data/train4/008_ship', f'../data/train4_resized_{IMG_SIZE}/008_ship')
resize_images_in_directory('../data/train4/009_truck', f'../data/train4_resized_{IMG_SIZE}/009_truck')
resize_images_in_directory('../data/train5/000_airplane', f'../data/train5_resized_{IMG_SIZE}/000_airplane')
resize_images_in_directory('../data/train5/001_automobile', f'../data/train5_resized_{IMG_SIZE}/001_automobile')
resize_images_in_directory('../data/train5/002_bird', f'../data/train5_resized_{IMG_SIZE}/002_bird')
resize_images_in_directory('../data/train5/003_cat', f'../data/train5_resized_{IMG_SIZE}/003_cat')
resize_images_in_directory('../data/train5/004_deer', f'../data/train5_resized_{IMG_SIZE}/004_deer')
resize_images_in_directory('../data/train5/005_dog', f'../data/train5_resized_{IMG_SIZE}/005_dog')
resize_images_in_directory('../data/train5/006_frog', f'../data/train5_resized_{IMG_SIZE}/006_frog')
resize_images_in_directory('../data/train5/007_horse', f'../data/train5_resized_{IMG_SIZE}/007_horse')
resize_images_in_directory('../data/train5/008_ship', f'../data/train5_resized_{IMG_SIZE}/008_ship')
resize_images_in_directory('../data/train5/009_truck', f'../data/train5_resized_{IMG_SIZE}/009_truck')

> We resize the images to 128x128 pixels to fit the VGG16 model.
> With this image size, we expect a 4 x 4 x 512 feature map after the convolutional base.

---
#### Group Datasets

In [ ]:
train_dirs = ['../data/train1_resized_128', '../data/train3_resized_128', '../data/train4_resized_128', '../data/train5_resized_128']
validation_dir = '../data/train2_resized_128'
test_dir = '../data/test_resized_128'

> ((2221985 + 2221986) % 5) + 1 = 2  
> Validation set: train2.  

---
####  Count Images in Categories

In [ ]:
def count_images_in_categories(directory):
    categories = os.listdir(directory)
    category_counts = {}
    for category in categories:
        category_counts[category] = len(os.listdir(os.path.join(directory, category)))
    return category_counts

train_counts_each_dir = [count_images_in_categories(train_dir) for train_dir in train_dirs]
validation_counts = count_images_in_categories(validation_dir)
test_counts = count_images_in_categories(test_dir)
train_counts = {category: sum([count.get(category, 0) for count in train_counts_each_dir]) for category in train_counts_each_dir[0]}

def plot_statistics(dataset_name, category_counts, color):
    categories = list(category_counts.keys())
    counts = list(category_counts.values())
    num_categories = len(categories)
    plt.figure(figsize=(8, 6))
    bars = plt.barh(range(num_categories), counts, color=color, alpha=1)

    for bar, count in zip(bars, counts):
        plt.text(bar.get_width() - 5, bar.get_y() + bar.get_height()/2, str(count), va='center', ha='right', color='white', fontweight='bold')

    plt.ylabel('Categories')
    plt.xlabel('Number of Images')
    plt.yticks(range(num_categories), categories)
    plt.title(f'Distribution of Images in {dataset_name} Dataset')
    plt.tight_layout()
    plt.show()

plot_statistics('Training Set', train_counts, 'blue')
plot_statistics('Validation Set', validation_counts, 'purple')
plot_statistics('Test Set', test_counts, 'red')

> We count the images of each category in each folder and plot the distribution.  
> We see that there are minor deviations the number of samples of each category in the train dataset and a bit more in the validation dataset. 

---
#### Create Datasets

In [ ]:
BATCH_SIZE = 256
NUM_CLASSES = len(train_counts)

train_datasets = [image_dataset_from_directory(directory, image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE) for directory in train_dirs]

class_names = train_datasets[0].class_names

train_dataset = train_datasets[0]
for dataset in train_datasets[1:]:
    train_dataset = train_dataset.concatenate(dataset)

train_dataset = train_dataset.shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE) # We tried without shuffle and prefetch
validation_dataset = image_dataset_from_directory(validation_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = image_dataset_from_directory(test_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

> We define the batch size of 256 and create an array with the label's names.  
> We create the train dataset by concatenating them, we shuffle the samples before each epoch and prefetch them to memory.  
> We do the same for the validation and test dataset except shuffling which is unnecessary.

---
#### Dataset Analysis

In [ ]:
for data_batch, labels_batch in train_dataset.take(1):
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)

> We have batches of 256 images, 128 by 128 pixels with 3 channels (RGB).  
> We also have batches of 256 labels, one for each image.  

---
#### Data Augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
        # We tried more techniques but the model didn't perform well
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomTranslation(0.1, 0.1),
        keras.layers.RandomContrast(0.1),
        keras.layers.RandomBrightness(0.1),
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomZoom(0.1),
    ]
)

> We define a data augmentation pipeline with horizontal flip, translation, contrast, brightness, rotation and zoom.

---
#### Augmented Dataset Visualization

In [ ]:
for images, labels in train_dataset.take(1):
    plt.figure(figsize=(128, 128))
    for i in range(len(images)):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(16, 16, i + 1)
        plt.imshow(augmented_images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i].numpy()])
        plt.axis("off")
        plt.tight_layout()
    plt.show()

> We print a random batch of augmented images from the train dataset along with their respective labels.  
> We see that the images are of different categories and are now of 128 x 128 pixels with slight variations.

---
#### Loading the VGG16 Model

In [ ]:
from keras.applications.vgg16 import VGG16
conv_base = VGG16(weights='imagenet', include_top=False)
conv_base.trainable = False

> We load the VGG16 model with the imagenet weights and without the top layer.  
> Setting trainable to False empties the list of trainable weights of the layer or model.  

---
#### Dense Network Arquitecture

In [ ]:
inputs = keras.Input(shape=(128, 128, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu", kernel_regularizer=regularizers.L2(0.01))(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

> We define a dense network with two hidden layers of 512 and 256 neurons respectively.
> We use the ReLU activation function and L2 regularization with a factor of 0.01.
> We also use dropout layers with 0.5 and 0.25 dropout rates.
> The output layer has the softmax activation function.
> We compile the model with the RMSprop optimizer and the sparse categorical crossentropy loss function.
> We also monitor the accuracy metric.
> We define the ReduceLROnPlateau and EarlyStopping callbacks.
> We train the model with 40 epochs and the validation dataset.
> We save the best model based on the validation loss.
> We save the training history.
> We evaluate the model with the validation dataset.
> We plot the loss and accuracy curves.
> We build the full model with the convolutional base and the dense network.
> We compile the full model with the RMSprop optimizer and the sparse categorical crossentropy loss function.
> We save the full model.
> We evaluate the full model with the test dataset.
> We predict the test dataset and collect the test labels and the predicted test labels.
> We plot the confusion matrix.
> We calculate the performance metrics.
> TODO:

---
#### Model Compilation

In [ ]:
initial_learning_rate = 0.001
optimizer = optimizers.Adam(learning_rate=initial_learning_rate)
loss_function = keras.losses.SparseCategoricalCrossentropy()
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)
early_stopping_callback = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='../models/model_t_data_augm_fine_tune_adam_1st_train.h5',
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

model.compile(optimizer=optimizer,
              loss=loss_function,
              metrics=['accuracy'])

> TODO:

---
#### Model Training

In [ ]:
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=(validation_dataset),
                    callbacks=[callbacks, lr_scheduler, early_stopping_callback])

> We train the model with 20 epochs and the validation dataset.  
> TODO:

---
#### Save Model History

In [ ]:
with open("../history/model_t_data_augm_fine_tune_adam_1st_train.pkl", "wb") as file:
    pickle.dump(history.history, file)

---
#### Model Evaluation

In [ ]:
val_loss, val_acc = model.evaluate(validation_dataset)
print('Model Validation Loss: ', val_loss)
print('Model Validation Accuracy: ', val_acc)

---
#### Model Training Visualization

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.tight_layout()
plt.show()

> TODO:

---
#### Transfer Learning - Fine Tuning

In [ ]:
# Freezing all layers except the last 4:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

> We unfreeze the last 4 layers of the convolutional base.
> TODO:

---
#### Model Compilation & Training

In [ ]:
model.compile(optimizer=optimizer,
              loss=loss_function,
              metrics=['accuracy'])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='../models/model_t_data_augm_fine_tune_adam_2nd_train.h5',
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=(validation_dataset),
                    callbacks=[callbacks, lr_scheduler, early_stopping_callback])

> We compile the model with the Adam optimizer and the sparse categorical crossentropy loss function.
> TODO:

---
#### Save Model History

In [ ]:
with open("../history/model_t_data_augm_fine_tune_adam_2nd_train.pkl", "wb") as file:
    pickle.dump(history.history, file)

---
#### Model Evaluation

In [ ]:
val_loss, val_acc = model.evaluate(validation_dataset)
print('Model Validation Loss: ', val_loss)
print('Model Validation Accuracy: ', val_acc)

---
#### Model Training Visualization

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.tight_layout()
plt.show()

> TODO:

---
#### Model Testing

In [ ]:
test_labels = []
test_predictions = []

for images, labels in test_dataset:
    test_labels.extend(labels.numpy())
    predictions = model.predict(images)
    test_predictions.extend(np.argmax(predictions, axis=-1))

test_labels = np.array(test_labels)
test_predictions = np.array(test_predictions)

---
#### Confusion Matrix

In [ ]:
cm = confusion_matrix(test_labels, test_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=90)
plt.show()

> TODO:

---
#### Performance Metrics

In [ ]:
acc = accuracy_score(y_true =  test_labels, y_pred = test_predictions)
print(f'Accuracy : {np.round(acc*100,2)}%')
precision = precision_score(y_true =  test_labels, y_pred = test_predictions, average='macro')
print(f'Precision - Macro: {np.round(precision*100,2)}%')
recall = recall_score(y_true =  test_labels, y_pred = test_predictions, average='macro')
print(f'Recall - Macro: {np.round(recall*100,2)}%')
f1 = f1_score(y_true =  test_labels, y_pred = test_predictions, average='macro')
print(f'F1-score - Macro: {np.round(f1*100,2)}%')
precision = precision_score(y_true =  test_labels, y_pred = test_predictions, average='weighted')
print(f'Precision - Weighted: {np.round(precision*100,2)}%')
recall = recall_score(y_true =  test_labels, y_pred = test_predictions, average='weighted')
print(f'Recall - Weighted: {np.round(recall*100,2)}%')
f1 = f1_score(y_true =  test_labels, y_pred = test_predictions, average='weighted')
print(f'F1-score - Weighted: {np.round(f1*100,2)}%')

> Accuracy is the proportion of correctly predicted instances out of the total instances.  
> Precision is the ratio of true positive predictions to the total predicted positives. Macro precision calculates this for each class independently and then averages them.  
> Weighted precision calculates the precision for each class, then averages them, weighted by the number of true instances for each class.  
> Recall is the ratio of true positive predictions to the total actual positives. Macro recall calculates this for each class independently and then averages them.  
> Weighted recall calculates the recall for each class, then averages them, weighted by the number of true instances for each class.  
> The F1-score is the harmonic mean of precision and recall. Macro F1-score calculates this for each class independently and then averages them.  
> Weighted F1-score calculates the F1-score for each class, then averages them, weighted by the number of true instances for each class.

---
# Conclusion
> We have trained a model with no data augmentation, dropout and RMSProp optimizer.  
> We opted to add Batch Normalization after each convolutional layer.  
> We have used L2 regularization on the convolutional and dense layers.  
> We have used dropout on the dense layers.  
> The model has shown some trouble distinguishing between some categories.  
> The model has shown some overfitting after 12 epochs but the best model was saved on the 28th epoch.  
> The model has shown an accuracy of 74.86% on the test set.  
> The model has shown a good performance on the test set with a macro F1-score of 74.86% and a weighted F1-score of 74.86%.  
> The model has shown a good performance on the test set with a macro precision of 74.86% and a weighted precision of 74.86%.  
> The model has shown a good performance on the test set with a macro recall of 74.86% and a weighted recall of 74.86%.  
> TODO: